In [ ]:
from h5py import File
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np 
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
f_eval = File('evaluations.hdf5', 'r')

In [ ]:
tables = [k for k in f_eval.keys()]
print(f"recorded runs: {tables}")

# tables = ['2'] # comment out if you want to use all tables

print(f"using tables {tables}")

# prepare data
data = None

for t in tables:
    dset = f_eval[t]
    if data is None:
        data = dset[...].copy()
    else:
        data = np.concatenate((data, dset[...]))
        
num_rows = len(data)
print(f"Found {num_rows} evaluations.")

In [ ]:
Y = data['y'].reshape(-1,1)
Y_std = data['y_std']
X = data['x']
steps = np.arange(len(Y))

In [ ]:
# normalize data somehow, e.g.
Y_normalized = (Y - np.mean(Y) )/ np.std(Y)
Y_std/np.std(Y)

# could fix likelihood noise to something like
noise = np.mean(Y_std/np.std(Y))
print(f"noise: {noise}")

In [ ]:
# TODO: Find subspaces ;)

import GPy
matern32 = GPy.kern.Matern32(input_dim=5, variance=noise*noise)

gp = GPy.models.GPRegression(X, Y_normalized, kernel=matern32)

gp.likelihood.variance.fix()
gp.optimize()

print(gp) # resulting lengthscale seems way to large... we found something more like 0.3
gp.log_likelihood() #

# TODO: Plot function along first two main components